## Creating data

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import glob
import sys
import json

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from tqdm import tqdm

### Read metadata

In [3]:
metadata = pd.read_csv('2020-03-13/all_sources_metadata_2020-03-13.csv')
metadata.shape

(29500, 14)

In [4]:
metadata.head()

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text
0,c630ebcdf30652f0422c3ec12a00b50241dc9bd9,CZI,Angiotensin-converting enzyme 2 (ACE2) as a SA...,10.1007/s00134-020-05985-9,NaN,32125455.0,cc-by-nc,NaN,2020,"Zhang, Haibo; Penninger, Josef M.; Li, Yimin; ...",Intensive Care Med,2.002765e+09,#3252,True
1,53eccda7977a31e3d0f565c884da036b1e85438e,CZI,Comparative genetic analysis of the novel coro...,10.1038/s41421-020-0147-1,NaN,NaN,cc-by,NaN,2020,"Cao, Yanan; Li, Lin; Feng, Zhimin; Wan, Shengq...",Cell Discovery,3.003431e+09,#1861,True
2,210a892deb1c61577f6fba58505fd65356ce6636,CZI,Incubation Period and Other Epidemiological Ch...,10.3390/jcm9020538,NaN,NaN,cc-by,The geographic spread of 2019 novel coronaviru...,2020,"Linton, M. Natalie; Kobayashi, Tetsuro; Yang, ...",Journal of Clinical Medicine,3.006065e+09,#1043,True
3,e3b40cc8e0e137c416b4a2273a4dca94ae8178cc,CZI,Characteristics of and Public Health Responses...,10.3390/jcm9020575,NaN,32093211.0,cc-by,"In December 2019, cases of unidentified pneumo...",2020,"Deng, Sheng-Qun; Peng, Hong-Juan",J Clin Med,1.776631e+08,#1999,True
4,92c2c9839304b4f2bc1276d41b1aa885d8b364fd,CZI,Imaging changes in severe COVID-19 pneumonia,10.1007/s00134-020-05976-w,NaN,32125453.0,cc-by-nc,NaN,2020,"Zhang, Wei",Intensive Care Med,3.006643e+09,#3242,False


In [10]:
metadata[metadata['has_full_text']==True].groupby(['source_x']).count()

,sha,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text
source_x,,,,,,,,,,,,,
CZI,149,149,149,0,40,149,105,149,148,149,139,149,149
PMC,12267,12267,11966,12267,12155,12267,11718,12207,12116,12267,0,0,12267
biorxiv,503,503,503,0,0,503,503,503,503,0,0,0,503
medrxiv,300,0,300,0,0,300,0,0,0,0,0,0,300


There are like 4 sources with lot of data:
- biorxiv_medrxiv
- comm_use_subset
- noncomm_use_subset
- pmc_custom_license

We are going use ```create corona.csv``` from https://www.kaggle.com/fmitchell259/create-corona-csv-file

In [54]:
sys.path.insert(0, '../')
root_path = '2020-03-13'

In [97]:
#empty df with covid papers format
corona_features = {"doc_id": [None], "source": [None], "title": [None], "abstract": [None], "text_body": [None]}
corona_df = pd.DataFrame(columns=corona_features)
corona_df

,doc_id,source,title,abstract,text_body


In [22]:
json_filenames = glob.glob("{}/**/*.json".format(root_path), recursive=True)
len(json_filenames)

13202

In [96]:
def return_corona_df(df, source):
    json_filenames = glob.glob("{}/{}/**/*.json".format(root_path, source), recursive=True)
    print("------------------------------------------------------------------------------------")
    print("Json files in source {}: {}".format(source, len(json_filenames)))
    
    for json_file in json_filenames:
        row = {"doc_id": None, "source": None, "title": None, "abstract": None, "text_body": None}
#         print(json_file)
        with open(json_file, 'r') as f:
            data = json.load(f)
            
            row['doc_id'] = data['paper_id']
            row['title'] = data['metadata']['title']
            
            abstract_list = [data['abstract'][x]['text'] for x in range(0,len(data['abstract']))]
            row['abstract'] = "\n ".join(abstract_list)
            
            body_list = []
            
            for _ in range(len(data['body_text'])):
                try:
                    body_list.append(data['body_text'][_]['text'])
                except:
                    pass
            
            
            row['text_body'] = "\n ".join(body_list)
            
            #adding to the dataframe
#             if source == 'b':
#                 row['source'] = 'biorxiv_medrxiv'
#             elif source == 'c':
#                 row['source'] = 'comm_use_subset'
#             elif source == 'n':
#                 row['source'] = 'noncomm_use_subset'
#             elif source == 'p':
#                 row['source'] = 'pmc_custom_license'
            row['source'] = source
#             print(row)
            df = df.append(row, ignore_index=True)
            
            
    return df

In [99]:
sources_list = ['biorxiv_medrxiv', 'comm_use_subset', 'noncomm_use_subset', 'pmc_custom_license']
pbar = tqdm(total=100)
for source in sources_list:
    
    corona_df = return_corona_df(corona_df, source)
    pbar.update(100/len(sources_list))
pbar.close()
    


  0%|          | 0/100 [00:00<?, ?it/s]

------------------------------------------------------------------------------------
Json files in source biorxiv_medrxiv: 803



 25%|██▌       | 25.0/100 [00:02<00:08,  8.75it/s]

------------------------------------------------------------------------------------
Json files in source comm_use_subset: 9000



 50%|█████     | 50.0/100 [01:03<00:40,  1.24it/s]

------------------------------------------------------------------------------------
Json files in source noncomm_use_subset: 1973



 75%|███████▌  | 75.0/100 [01:16<00:17,  1.40it/s]

------------------------------------------------------------------------------------
Json files in source pmc_custom_license: 1426



100%|██████████| 100.0/100 [01:23<00:00,  1.20it/s]


In [100]:
data_folder = 'data/input'
corona_df.to_csv("{}/corona")

,doc_id,source,title,abstract,text_body
0,f905f78b32f63c6d14a79984dfb33f1b358b8ab4,biorxiv_medrxiv,Multimerization of HIV-1 integrase hinges on c...,New anti-AIDS treatments must be continually d...,"In the absence of a curative treatment, the hi..."
1,abcfffafab399149d4adadd6bb458c4994e2025d,biorxiv_medrxiv,Time-varying transmission dynamics of Novel Co...,Rationale: Several studies have estimated basi...,"Eighteen years ago, severe acute respiratory s..."
2,0cb9c296684ca5e71462d825cab2827854a01544,biorxiv_medrxiv,p53 is not necessary for DUX4 pathology,Summary Statement: DUX4 is thought to mediate ...,Facioscapulohumeral muscular dystrophy (FSHD) ...
3,9bbfd3d34ee18ea1b9f4669331a6cee9c5992893,biorxiv_medrxiv,Virological assessment of hospitalized cases o...,"emerged in late 2019 1,2 . Initial outbreaks i...",Pharyngeal virus shedding was very high during...
4,1218f278a4f8d83dac14b23c8f698062812ef9d5,biorxiv_medrxiv,Potential impact of seasonal forcing on a SARS...,A novel coronavirus (SARS-CoV-2) first detecte...,(2.2 with 90% high density interval 1.4-3.8 (R...
5,a59906b732bf4a489e282c3e4f499d4166c622e7,biorxiv_medrxiv,Evaluating performance of metagenomic characte...,"Background 3\n In silico bacterial, viral, and...","Background 3\n In silico bacterial, viral, and..."
6,baabfb35a321ea12028160e0d2c1552a2fda2dd5,biorxiv_medrxiv,Clinical Features of COVID-19-Related Liver Da...,with rapidly increasing the number of cases (n...,"Since December 2019, a novel coronavirus was i..."
7,21100aba41a4bfb48d7dc37f1bf5dbb38bf3867a,biorxiv_medrxiv,Network-based Drug Repurposing for Human Coron...,"Human Coronaviruses (HCoVs), including severe ...",Coronaviruses (CoVs) typically affect the resp...
8,7852aafdfb9e59e6af78a47af796325434f8922a,biorxiv_medrxiv,Detectable serum SARS-CoV-2 viral load (RNAaem...,Although the SARS-CoV-2 viral load detection o...,"The novel coronavirus disease , which is cause..."
9,317215c463463ddb39fa7564f67239dc484b86a9,biorxiv_medrxiv,Projecting the end of the Zika virus epidemic ...,International Concern following a cluster of a...,"Starting as early as 2013, 1 has since spread ..."
